In [1]:
def print_time():
    from datetime import datetime
    print("%s min %s s"%(datetime.now().minute,datetime.now().second))

In [2]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import socket
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [3]:
print_time()

11 min 11 s


In [4]:
ip           = socket.gethostname()
port         = 12066
nb_of_second = 5

# 1) création du spark context

In [5]:
# Suppression du SparkContext s’il existe déjà
try: del ssc
except: pass
ssc             = StreamingContext(sc, batchDuration=nb_of_second)

# 2)  Fonction d'update de valeurs

In [6]:
def update_achats(nouvelles_valeurs, valeur_actuelle ):
    if valeur_actuelle is None:
        valeur_actuelle = 0
    return sum(nouvelles_valeurs) + valeur_actuelle

# 3) définition du DAG

In [7]:
lignes_de_texte   = ssc.socketTextStream(ip,port )
entreprise_nombre = lignes_de_texte.map(lambda ligne:ligne.split(","))
paired_rdd        = entreprise_nombre.map(lambda x: (x[0], float(x[1])))
# Somme des achats par clef 
sommer_les_achats = paired_rdd.updateStateByKey(lambda x: x)
# affichage
print_time()
sommer_les_achats.pprint()

11 min 13 s


In [8]:
ssc.checkpoint("./checkpoint")

In [9]:
print("start on  %s:%s"%(ip, port))
ssc.start()

start on  2b718a58c083:12066


In [10]:
if False:
    ssc.stop() 

In [11]:
!ls -l ./checkpoint/

total 80
drwxr-xr-x 4 jovyan users  128 Apr 24 20:55 002fdde4-5af6-4789-b139-bcf257cacfe5
drwxr-xr-x 4 jovyan users  128 Apr 24 20:54 00eee579-0728-4607-bf49-4b00e3ada5d4
drwxr-xr-x 2 jovyan users   64 Apr 24 21:07 039ecb00-406f-45d6-ab97-04dfa13250ed
drwxr-xr-x 4 jovyan users  128 Apr 25 07:50 18250785-02ea-4caf-84c4-085310cf8700
drwxr-xr-x 2 jovyan users   64 Apr 25 21:10 1936aa4a-ebcb-44b2-b51b-f6b21475aadf
drwxr-xr-x 2 jovyan users   64 Apr 25 21:06 29718281-024d-4e1b-b35c-1e6a8f88869f
drwxr-xr-x 2 jovyan users   64 Apr 25 07:51 6615678f-3b3c-4c35-9aba-5ad62f4fd272
drwxr-xr-x 2 jovyan users   64 Apr 25 21:06 7f44c4eb-5814-41b4-a75d-ff5502ffc108
drwxr-xr-x 4 jovyan users  128 Apr 24 21:06 89173ea6-26a8-45b8-9ec0-35833e46eb83
drwxr-xr-x 2 jovyan users   64 Apr 25 21:11 9c8da2bf-6a59-4205-bcee-faaae3b476f2
drwxr-xr-x 2 jovyan users   64 Apr 25 08:05 a1657814-17fe-4f48-9ad3-abd18f88810e
-rw-r--r-- 1 jovyan users 7159 Apr 25 21:08 checkpoint-1556226480000
-rw-r--r-- 1 jovyan users 7163 